<a href="https://colab.research.google.com/github/sresthabugu/Sentimental-and-toxicity/blob/main/tox_hindisen3indi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas scikit-learn datasets transformers torch
!pip install pandas_ods_reader

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from datasets import DatasetDict, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving hi_3500.ods to hi_3500.ods


In [ ]:
from pandas_ods_reader import read_ods

In [ ]:
data = read_ods("hi_3500.ods",1, headers=False)
data.head()

,column.0,column.1
0,लोग वतन तक खा जाते हैं इसका इसे यकीन नहींमान ज...,negative
1,गुमनाम है वतन पर मिटने वाले लोग आतन्कवादियों स...,negative
2,ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है,negative
3,यूपी में बड़े स्तर पर दंगे करवा सकती है बीजेपी...,negative
4,अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा...,negative


In [ ]:
data = data.sample(n=1000)


In [ ]:
data['column.1'].unique()

array(['negative', 'positive', 'neutral'], dtype=object)

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data['column.1'] = label_encoder.fit_transform(data['column.1'])

In [ ]:
data['column.1'].unique()

array([2, 0, 1])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['column.0'], data['column.1'], test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
df_train = pd.DataFrame()
df_test = pd.DataFrame()
df_val = pd.DataFrame()

In [ ]:
df_train['texts'] = X_train
df_train['label'] = y_train
df_test['texts'] = X_test
df_test['label'] = y_test
df_val['texts'] = X_val
df_val['label'] = y_val

In [ ]:
import datasets
from datasets import load_dataset, DatasetDict
utube_sent = DatasetDict()
utube_sent["train"] = datasets.Dataset.from_pandas(df = df_train)
utube_sent["test"] = datasets.Dataset.from_pandas(df = df_test)
utube_sent['val'] = datasets.Dataset.from_pandas(df = df_val)
utube_sent

DatasetDict({
    train: Dataset({
        features: ['texts', 'label', '__index_level_0__'],
        num_rows: 640
    })
    test: Dataset({
        features: ['texts', 'label', '__index_level_0__'],
        num_rows: 200
    })
    val: Dataset({
        features: ['texts', 'label', '__index_level_0__'],
        num_rows: 160
    })
})

In [ ]:
utube_sent = utube_sent.rename_column("label","labels")
utube_sent = utube_sent.rename_column("texts","text")
utube_sent = utube_sent.remove_columns("__index_level_0__")


utube_sent

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 640
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 200
    })
    val: Dataset({
        features: ['text', 'labels'],
        num_rows: 160
    })
})

In [ ]:
checkpoint = 'ai4bharat/indic-bert'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)


In [ ]:
tokenized_datasets = utube_sent.map(tokenize_function, batched=True)


Map:   0%|          | 0/640 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val'],
    tokenizer=tokenizer
)

In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,0.975903
2,No log,0.718722
3,No log,0.695224


TrainOutput(global_step=120, training_loss=0.8680609385172526, metrics={'train_runtime': 28.5118, 'train_samples_per_second': 67.341, 'train_steps_per_second': 4.209, 'total_flos': 5149056240576.0, 'train_loss': 0.8680609385172526, 'epoch': 3.0})

In [ ]:
trainer.evaluate()


{'eval_loss': 0.6952241063117981,
 'eval_runtime': 0.5974,
 'eval_samples_per_second': 267.81,
 'eval_steps_per_second': 16.738,
 'epoch': 3.0}

In [ ]:

model.save_pretrained('./saved_model')
tokenizer.save_pretrained('./saved_model')

('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/spiece.model',
 './saved_model/added_tokens.json',
 './saved_model/tokenizer.json')

In [ ]:
predictions = trainer.predict(tokenized_datasets['test'])
preds = torch.nn.functional.softmax(torch.tensor(predictions.predictions), dim=-1)
df_test['toxicity_score'] = preds[:, 1].tolist()


In [ ]:
df_test.to_csv('test_results_with_toxicity_scores.csv', index=False)


In [ ]:
print(df_test.head())


                                                  texts  label  toxicity_score
6408  ये सब कुछ उनकी खराब फिटनेस व आए दिन चोटिल होने...      0        0.049931
164   BJP के नेतृत्व में AAP और लोकतंत्र के खिलाफ ये...      0        0.196118
4573     लोगों ने ऑफर व डिस्काउंट का भरपूर फायदा उठाया।      2        0.049422
4917  कोर्ट का मानना है कि कोर्ट का समय बर्बाद करने ...      2        0.048721
2638  RT @aajtak: .@crpfindia के 80वें स्थापना दिवस ...      1        0.852152
